이번 챕터에서는 파이토치로 인공 신경망을 이용하여 태깅 작업을 하는 모델을 만듭니다. 개체명 인식기와 품사 태거를 만드는데, 이러한 두 작업의 공통점은 many-to-many RNN이면서 앞, 뒤 시점의 입력을 모두 참고하는 양방향 RNN을 사용한다는 것입니다. 

실습을 하기 전에 먼저 어떻게 실습이 진행될지 전반적으로 정리하곘습니다.

## **1. 훈련 데이터에 대한 이해**

태깅 작업은 지도 학습에 속합니다. 이 챕터에서는 태깅을 해야하는 단어 데이터를 X, 레이블에 해당되는 태깅 정보 데이터는 y라고 하겠습니다. 이번 챕터에서 X와 y 데이터의 쌍은 병렬 구조를 가진다는 특징이 있습니다. X와 y의 샘플 길이는 같습니다. 예를 들어 품사 태깅 작업을 한다고 가정했을 때 다음과 같은 구조를 가집니다.

||X_train|y_train|length|
|:-:|:-:|:-:|:-:|
|0|['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb']|['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O']|8|
|1|['peter', 'blackburn']|['B-PER', 'I-PER']|2|
|2|	['brussels', '1996-08-22' ]|['B-LOC', 'O']|2|
|3|['The', 'European', 'Commission']|['O', 'B-ORG', 'I-ORG']|3|

가령, X_train[3]의 'The'와 y_train[3]의 'O'는 하나의 쌍입니다. 이렇게 병렬 관계를 가지는 각 데이터는 정수 인코딩 과정을 거친 후, 모든 데이터의 길이를 동일하게 맞춰 주기 위한 패딩(padding) 작업을 거칩니다.

## **2. 시퀀스 레이블링**

위와 같이 입력 시퀀스 X = $[x_1, x_2, x_3, \cdots, x_n]$ 에 대하여 레이블 시퀀스 y = $[y_1, y_2, y_3, \cdots, y_n]$ 를 각각 부여하는 작업을 시퀀스 레이블링(Sequence Labeling Task)이라고 합니다. 태깅 작업은 대표적인 시퀀스 레이블링 작업입니다.

## **3. RNN의 many-to-many 문제**

이제 RNN이 어떻게 설계되는지 확인해보겠습니다. 예를 들어 위에서 설명한 데이터 중 첫번째 데이터에 해당되는 X_train[0]를 가지고 4번의 시점까지 RNN을 진행하였을 때 그림은 다음과 같습니다.

<img src = "https://wikidocs.net/images/page/33805/forwardrnn_ver2.PNG">

그러나 이번 실습에서는 양방향 RNN을 사용하므로 다음과 같습니다.

<img src = "https://wikidocs.net/images/page/33805/bidirectionalrnn_ver2.PNG">